In [1]:
import pandas as pd
import spacy

In [2]:
nlp = spacy.load('en_core_web_md')

In [7]:
df_demo = pd.read_csv('demo.csv',index_col=0)

In [8]:
df_demo.head()

,class,date,description,title
0,Disasters and accidents,2017/9/1,"Hurricane Irma, now a Category 2 hurricane wit...",2017 Atlantic hurricane season
1,Health and medicine,2017/9/1,"Researchers report, in the Environmental Scien...",Great Lakes
2,International relations,2017/9/1,South Korean President Moon Jae-in and U.S. Pr...,South Korea鈥揢nited States relations
3,Law and crime,2017/9/1,United States federal judge Richard Posner has...,United States Court of Appeals for the Seventh...
4,Politics and elections,2017/9/1,"The Labour Party, led by Jacinda Ardern, surge...","New Zealand general election, 2017"


In [12]:
def class_code(type_str):
    type_str = type_str.lower()
    if 'armed' in type_str or 'attack' in type_str or 'conflict' in type_str:
        return 1
    elif 'disaster' in type_str or 'accident' in type_str:
        return 2
    elif 'law' in type_str or 'crime' in type_str:
        return 3
    elif 'politic' in type_str or 'election' in type_str:
        return 4
    elif 'international' in type_str or 'relation' in type_str:
        return 5
    elif 'science' in type_str or 'technology' in type_str:
        return 6
    elif 'business' in type_str or 'econom' in type_str:
        return 7
    elif 'art' in type_str or 'culture' in type_str:
        return 8
    elif 'sport' in type_str:
        return 9
    elif 'health' in type_str or 'environment' in type_str:
        return 10
    else:
        return 0

In [13]:
df_demo['class_code'] = df_demo['class'].apply(class_code)

In [18]:
df_demo.head()

,class,date,description,title,class_code
0,Disasters and accidents,2017/9/1,"Hurricane Irma, now a Category 2 hurricane wit...",2017 Atlantic hurricane season,2
1,Health and medicine,2017/9/1,"Researchers report, in the Environmental Scien...",Great Lakes,10
2,International relations,2017/9/1,South Korean President Moon Jae-in and U.S. Pr...,South Korea鈥揢nited States relations,5
3,Law and crime,2017/9/1,United States federal judge Richard Posner has...,United States Court of Appeals for the Seventh...,3
4,Politics and elections,2017/9/1,"The Labour Party, led by Jacinda Ardern, surge...","New Zealand general election, 2017",4


In [42]:
def description_clean(description):
    description = description.split('. (')[0]+'.'
    return description

In [43]:
df_demo['des_clean'] = df_demo['description'].apply(description_clean)

In [57]:
from spacy import displacy
from IPython.display import display, HTML

In [122]:
def class_similarity(class_text,span):
    return nlp(class_text).similarity(nlp(span))

In [123]:
def print_pos_(doc,class_text):
    doc_pos_ = []
    for token in doc:
        if token.pos_ in ['VERB']:   #['AUX','NOUN','PROPN','VERB']:
            doc_pos_.extend([token.text,token.pos_,str(class_similarity(class_text,token.text))])
    print('|'.join(doc_pos_))

In [124]:
def print_noun_chunks(doc,class_text):
    noun_chunks =[]
    for i in doc.noun_chunks:
        noun_chunks.extend([i.text,str(class_similarity(class_text,i.text))])
    print('|'.join(noun_chunks))

In [148]:
def expand_with_noun_chuncks(noun_chuncks,ent):
    for noun_chunck in noun_chuncks:
        if ent in noun_chunck:
            return noun_chunck
    return ent

In [205]:
def argument_candidate(doc,class_text):
    arguments = []
    noun_chuncks = [i.text for i in doc.noun_chunks]
    for i in doc.ents:
        #arguments.append((expand_with_noun_chuncks(noun_chuncks,i.text),i.label_,str(class_similarity(class_text,expand_with_noun_chuncks(noun_chuncks,i.text)))))
        arguments.append((i.text,i.label_,str(class_similarity(class_text,i.text))))
    related_when = [i for i in arguments if i[1] in ['TIME','DATE']]
    related_where = [i for i in arguments if i[1] in ['GPE','LOC','FACILITY']]
    related_who = [i for i in arguments if i[1] in ['PERSON','NORP','ORG','']]
    related_what = [i for i in arguments if i[1] in ['PRODUCT','EVENT','WORK_OF_ART','LAW','LANGUAGE','PERCENT','MONEY','QUANTITY','ORDINAL','CARDINAL']]
    return arguments,{
        'related_when':related_when,
        'related_where':related_where,
        'related_who':related_who,
        'related_what':related_what,
    }

In [207]:
def trigger_candidate(doc,arguments,class_text):
    triggers = []
    arguments = [argu[0] for argu in arguments]
    noun_chuncks = [i.text for i in doc.noun_chunks]
    triggers.extend([(i,str(class_similarity(class_text,i))) for i in set(noun_chuncks)-set(arguments)])
    for token in doc:
        if token.tag_.startswith('V'):
            triggers.append((token.text,token.tag_,str(class_similarity(class_text,token.text))))
    return triggers

In [224]:
def print_arguments_triggers(arguments_dict,triggers):
    print('Arguments:\n')
    #pdb.set_trace()
    for k,v in arguments_dict.items():
        print(k+' : '+'|'+'|'.join(['|'.join(i) for i in v]))
    triggers_str = ''
    for i in triggers:
        triggers_str+='|'+'|'.join(i)
    print('Triggers:\n',triggers_str)   

In [225]:
def display_doc(doc,style='ent',class_text=None):
    doc = nlp(doc)
    display(HTML(displacy.render(doc, style=style)))
    arguments,arguments_dict = argument_candidate(doc,class_text)
    triggers = trigger_candidate(doc,arguments,class_text)
    print_arguments_triggers(arguments_dict,triggers)
    #print_pos_(doc,class_text)
    #print_noun_chunks(doc,class_text)

In [226]:
for event in df_demo.iterrows():
    print(event[0])
    print(event[1]['class'],'\t',event[1]['date'],'\t',event[1]['title'])
    display_doc(event[1]['des_clean'],class_text=event[1]['class'].replace('and',''))

0
Disasters and accidents 	 2017/9/1 	 2017 Atlantic hurricane season


Arguments:

related_when : |this weekend|DATE|0.283249331654|next Thursday|DATE|0.160565826671
related_where : |the Leeward Islands|LOC|0.247873127015|the Caribbean|LOC|0.208435926875
related_who : |Hurricane Irma|PERSON|0.357908785797
related_what : |now a Category 2 hurricane|QUANTITY|0.380235448992|110 miles per hour|QUANTITY|0.185376154623|175 kilometers per hour|QUANTITY|0.148967080751
Triggers:
 |it|0.264386930191|the danger|0.462647103084|110 miles|0.13337395046|maximum sustained winds|0.374320373452|hour|0.162428970276|175 kilometers|0.0666239555169|is|VBZ|0.168897048868|expected|VBN|0.298831563871|resume|VB|0.146153933788|strengthening|VBG|0.212357447734|increasing|VBG|0.360947489227|nears|VBZ|0.138839396512
1
Health and medicine 	 2017/9/1 	 Great Lakes


Arguments:

related_when : |
related_where : |the Niagara River|LOC|0.388616151142|the Great Lakes|LOC|0.358734376058|Lakes Erie|LOC|0.32894182904|Ontario|GPE|0.32870366923
related_who : |the Environmental Science & Technology journal|ORG|0.626985301713
related_what : |10 kinds|CARDINAL|0.338455982602
Triggers:
 |antidepressant concentrations|0.621126117878|Speculated causes|0.402148572707|water|0.354915577487|Researchers|0.486778847535|inadequate treatment|0.606032998365|said bodies|0.362389242075|fish|0.307684981229|the discovery|0.409114996831|report|VBP|0.333384865889|links|VBZ|0.247388982788|Speculated|VBN|0.190679936706|include|VBP|0.338148698629|introduced|VBN|0.260558532936|said|VBN|0.247601628986
2
International relations 	 2017/9/1 	 South Korea鈥揢nited States relations


Arguments:

related_when : |
related_where : |U.S. President Donald Trump|GPE|0.413748916657|South Korean President Moon Jae-in|GPE|0.40294889442|South Korea's missile development|GPE|0.51063238924
related_who : |South Korean President Moon Jae-in|NORP|0.40294889442|South Korean President Moon Jae-in|PERSON|0.40294889442|U.S. President Donald Trump|PERSON|0.413748916657
related_what : |
Triggers:
 |the South Korea Ballistic Missile Range Guidelines|0.486171632835|agree|VBP|0.277149975514|revise|VB|0.307453127518|caps|VBZ|0.0916833181096
3
Law and crime 	 2017/9/1 	 United States Court of Appeals for the Seventh Circuit


Arguments:

related_when : |three decades|DATE|0.423120644064|1981|DATE|0.188043900903
related_where : |United States federal judge Richard Posner|GPE|0.631050195264|Chicago|GPE|0.29550988158|Illinois|GPE|0.342008780656
related_who : |United States federal judge Richard Posner|PERSON|0.631050195264|U.S. Circuit Court of Appeals|ORG|0.588310700173|President Ronald Reagan|PERSON|0.265534860052|United States federal judge Richard Posner|PERSON|0.631050195264|the bench.(AP|ORG|0.403803427413
related_what : |the 7th U.S. Circuit Court|ORDINAL|0.529322510022|more than 3,300 opinions|CARDINAL|0.398155457616
Triggers:
 |Appeals|0.462955921122|his retirement|0.423290169946|service|0.284943002868|Justice Posner|0.45301231031|the bench|0.343527492927|has|VBZ|0.404189224658|announced|VBN|0.185691645479|headquartered|VBN|0.230955994087|Appointed|VBN|0.292058855466|has|VBZ|0.404189224658|written|VBN|0.363548749008
4
Politics and elections 	 2017/9/1 	 New Zealand general election, 2017


...

Arguments:

related_when : |
related_where : |Bengaluru|GPE|0.0651254167868
related_who : |Prominent Indian journalist|NORP|0.42397370554|Lankesh Patrike editor|PERSON|0.17564787189|Gauri Lankesh|PERSON|0.0
related_what : |
Triggers:
 |the city|0.500354653334|unidentified men|0.388254146727|her house|0.409643719279|is|VBZ|0.422295721413|shot|VBN|0.265817437951
33
Law and crime 	 2017/9/5 	 Australian Marriage Law Postal Survey (Same-sex marriage)


Arguments:

related_when : |
related_where : |
related_who : |The High Court of Australia|ORG|0.587607746366|the Australian Marriage Law Postal Survey|NORP|0.711241499187
related_what : |(The Guardian)|WORK_OF_ART|0.361361944156
Triggers:
 |Australia|0.25587200476|(The Guardian|0.413507017033|The High Court|0.593857575155|a complaint|0.515442505477|the postal survey|0.453356289133|receives|VBZ|0.184443863222|citing|VBG|0.33119984825
34
Politics and elections 	 2017/9/5 	 Politics of Taiwan


Arguments:

related_when : |
related_where : |Tainan Mayor William Lai Ching-te|GPE|0.269547018921|the Republic of China|GPE|0.469497943587
related_who : |Tainan Mayor William Lai Ching-te|PERSON|0.269547018921|Tsai Ing-wen|PERSON|0.1266557292
related_what : |
Triggers:
 |the Republic|0.534595955881|China|0.156207825947|the new Premier|0.332105353284|the President|0.521588055935|is|VBZ|0.271034264143|appointed|VBN|0.302930283368
35
Politics and elections 	 2017/9/5 	 Deferred Action for Childhood Arrivals


Arguments:

related_when : |the next six months|DATE|0.338907360369
related_where : |the U.S.|GPE|0.437061635613
related_who : |The Trump Administration|ORG|0.591466005224|the DACA program|ORG|0.280110416205|Congress|ORG|0.57855267145
related_what : |about 800,000 people|CARDINAL|0.327572046668
Triggers:
 |it|0.359504523576|who|0.298166567757|the situation|0.45906628286|children|0.219449375835|legislation|0.497035700057|the deportation|0.395922391098|announces|VBZ|0.216135332413|be|VB|0.313054534805|ending|VBG|0.300788915757|has|VBZ|0.305889900052|halted|VBN|0.26774693886|were|VBD|0.29053448751|brought|VBN|0.31831567405|is|VBZ|0.271034264143|called|VBN|0.245750025744|pass|VB|0.185807649204|correct|VB|0.251773396354
36
Sports 	 2017/9/5 	 2017鈥?8 NBA season


Arguments:

related_when : |
related_where : |
related_who : |Billionaire investor Tilman Fertitta|PERSON|0.11165074302|the Houston Rockets NBA basketball team|ORG|0.599674816765
related_what : |US$2.2 billion|MONEY|0.0792160550851
Triggers:
 |buys|VBZ|0.19532524527


In [314]:
from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer
from collections import defaultdict

In [317]:
def efitf(X):
    count = CountVectorizer()
    X_train_count = count.fit_transform(X)
    tfidf = TfidfTransformer(use_idf=True)
    X_train_tfidf = tfidf.fit_transform(X_train_count)
    tf_feature_names = count.get_feature_names()
    X_train_tfidf = [list(i) for i in list(X_train_tfidf.toarray())]
    EFITF = defaultdict(list)
    for Type,values in enumerate(X_train_tfidf):
        #pdb.set_trace()
        for index,value in enumerate(values):
            if value > 0.0:
                EFITF[Type].append({tf_feature_names[index]:value}) 
    return EFITF

In [229]:
df_demo.head()

,class,date,description,title,class_code,des_clean
0,Disasters and accidents,2017/9/1,"Hurricane Irma, now a Category 2 hurricane wit...",2017 Atlantic hurricane season,2,"Hurricane Irma, now a Category 2 hurricane wit..."
1,Health and medicine,2017/9/1,"Researchers report, in the Environmental Scien...",Great Lakes,10,"Researchers report, in the Environmental Scien..."
2,International relations,2017/9/1,South Korean President Moon Jae-in and U.S. Pr...,South Korea鈥揢nited States relations,5,South Korean President Moon Jae-in and U.S. Pr...
3,Law and crime,2017/9/1,United States federal judge Richard Posner has...,United States Court of Appeals for the Seventh...,3,United States federal judge Richard Posner has...
4,Politics and elections,2017/9/1,"The Labour Party, led by Jacinda Ardern, surge...","New Zealand general election, 2017",4,"The Labour Party, led by Jacinda Ardern, surge..."


In [233]:
df_demo[df_demo['class_code'] == 1]['des_clean'].tolist()

['Seven people are killed and 13 are injured after suicide bombers hit a state-run power station near the northern city of Samarra, Iraq. The Islamic State of Iraq and the Levant claims responsibility for the attack.',
 'Syrian state television reports the Syrian Army reaches a point 3 kilometers from Deir ez-Zor, a city besieged by the Islamic State of Iraq and the Levant since 2014.',
 'Siege of Deir ez-Zor (2014鈥?7)\r\n\r\nThe Syrian Army lifts the 28-month-long siege of Deir ez-Zor by the Islamic State of Iraq and the Levant.',
 'More than 123,000 Rohingya refugees have fled Myanmar and crossed into Bangladesh due to escalating violence by the Myanmar Army.']

In [239]:
X = [0]*11

In [242]:
for i in range(11):
    X[i] = ' '.join(df_demo[df_demo['class_code'] == i]['des_clean'].tolist())

In [318]:
EFITF =  efitf(X)

In [415]:
doc = nlp('The Cambodia Daily newspaper announces it will publish its final edition after being ordered to pay million in taxes as result of an investigation initiated by Prime Minister of Cambodia Hun Sen into private companies operating in Cambodia')

In [329]:
doc

The Cambodia Daily newspaper announces it will publish its final edition after being ordered to pay USD$6.3 million in taxes as a result of an investigation, initiated by Prime Minister of Cambodia Hun Sen , into private companies operating in Cambodia .

In [332]:
doc.ents

(The Cambodia Daily newspaper, USD$6.3 million, Cambodia, Hun Sen, Cambodia)

In [338]:
ents_texts = [i.text for i in doc.ents]

In [419]:
[i.text for i in doc.noun_chunks if i.text not in ents_texts and not i.is_stop]

AttributeError: 'spacy.tokens.span.Span' object has no attribute 'is_stop'

### efitf vs doc2vec